## PROYECTO 13
---

### 1. Descripción del proyecto 
---
La cadena de gimnasios Model Fitness está desarrollando una estrategia de interacción con clientes basada en datos analíticos.

Uno de los problemas más comunes que enfrentan los gimnasios y otros servicios es la pérdida de clientes. ¿Cómo descubres si un/a cliente ya no está contigo? Puedes calcular la pérdida en función de las personas que se deshacen de sus cuentas o no renuevan sus contratos. Sin embargo, a veces no es obvio que un/a cliente se haya ido: puede que se vaya de puntillas.

Los indicadores de pérdida varían de un campo a otro. Si un usuario o una usuaria compra en una tienda en línea con poca frecuencia, pero con regularidad, no se puede decir que ha huido. Pero si durante dos semanas no ha abierto un canal que se actualiza a diario, es motivo de preocupación: es posible que tu seguidor o seguidor/a se haya aburrido y te haya abandonado.

Con el fin de combatir la cancelación, Model Fitness ha digitalizado varios de sus perfiles de clientes. Tu tarea consiste en analizarlos y elaborar una estrategia de retención de clientes.

### 2. Objetivos 
---

- Predecir la probabilidad de pérdida (para el próximo mes) para cada cliente.
- Elaborar retratos de usuarios típicos: selecciona los grupos más destacados y describe sus características principales.
- Analizar los factores que más impactan la pérdida.
- Sacar conclusiones básicas y elaborar recomendaciones para mejorar la atención al cliente:

    *identificar a los grupos objetivo;
    
   *sugerir medidas para reducir la rotación;
   
describir cualquier otro patrón que observes con respecto a la interacción con los clientes.

### 3.Análisis exploratorio de datos (EDA)
---

In [1]:
# Importación de librerías 
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [13]:
# Asignación de varible y visualización 
churn_df= pd.read_csv('gym_churn_us.csv', encoding='unicode_escape')
churn_df.info()
churn_df.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   gender                             4000 non-null   int64  
 1   Near_Location                      4000 non-null   int64  
 2   Partner                            4000 non-null   int64  
 3   Promo_friends                      4000 non-null   int64  
 4   Phone                              4000 non-null   int64  
 5   Contract_period                    4000 non-null   int64  
 6   Group_visits                       4000 non-null   int64  
 7   Age                                4000 non-null   int64  
 8   Avg_additional_charges_total       4000 non-null   float64
 9   Month_to_end_contract              4000 non-null   float64
 10  Lifetime                           4000 non-null   int64  
 11  Avg_class_frequency_total          4000 non-null   float

,gender,Near_Location,Partner,Promo_friends,Phone,Contract_period,Group_visits,Age,Avg_additional_charges_total,Month_to_end_contract,Lifetime,Avg_class_frequency_total,Avg_class_frequency_current_month,Churn
846,0,0,0,0,1,1,0,30,155.025211,1.0,3,1.032163,0.943424,0
1801,1,0,1,0,1,1,0,23,216.591038,1.0,2,1.468258,0.290798,1
3432,0,1,0,0,1,1,0,26,109.549299,1.0,0,1.634142,0.550881,1
200,0,1,1,0,1,1,0,34,46.492200,1.0,0,1.010771,0.000000,1
1453,0,1,1,1,1,6,0,34,214.339904,6.0,10,1.179269,1.062388,0
1713,1,1,0,0,1,1,0,22,31.396454,1.0,1,2.910890,2.765920,1
1204,1,0,1,0,1,1,0,24,127.142083,1.0,0,2.289697,1.701966,1
1264,1,1,0,0,1,1,1,29,71.523522,1.0,1,2.729743,2.753885,0
3752,1,1,1,1,1,12,0,30,234.651342,10.0,4,2.733622,2.754167,0
2704,1,1,0,0,1,6,1,26,202.994232,6.0,10,2.917560,3.083981,0


In [20]:
churn_df.rename(columns={'gender':'Gender'}, inplace=True)
churn_df['Month_to_end_contract']= churn_df['Month_to_end_contract'].astype('int')

In [15]:
churn_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Gender,4000.0,0.510250,0.499957,0.000000,0.000000,1.000000,1.000000,1.000000
Near_Location,4000.0,0.845250,0.361711,0.000000,1.000000,1.000000,1.000000,1.000000
Partner,4000.0,0.486750,0.499887,0.000000,0.000000,0.000000,1.000000,1.000000
Promo_friends,4000.0,0.308500,0.461932,0.000000,0.000000,0.000000,1.000000,1.000000
Phone,4000.0,0.903500,0.295313,0.000000,1.000000,1.000000,1.000000,1.000000
Contract_period,4000.0,4.681250,4.549706,1.000000,1.000000,1.000000,6.000000,12.000000
Group_visits,4000.0,0.412250,0.492301,0.000000,0.000000,0.000000,1.000000,1.000000
Age,4000.0,29.184250,3.258367,18.000000,27.000000,29.000000,31.000000,41.000000
Avg_additional_charges_total,4000.0,146.943728,96.355602,0.148205,68.868830,136.220159,210.949625,552.590740
Month_to_end_contract,4000.0,4.322750,4.191297,1.000000,1.000000,1.000000,6.000000,12.000000
